# A Guide to Market Basket Analysis in PySpark 🛒

This tutorial introduces **frequent itemset mining** and **association rule mining**, two fundamental techniques for uncovering meaningful relationships within large datasets. A classic example is "market basket analysis," where we analyze customer transactions to identify products that are frequently purchased together. For instance, we might find that customers who buy tortilla chips also tend to buy salsa. We'll cover two algorithms:

1.  **A Priori**: A classic algorithm that we'll build from scratch to understand the core logic.
2.  **FP-Growth**: A modern, highly scalable algorithm that is built directly into Spark's Machine Learning library (MLlib).

**Credits**

This tutorial is adapted from [MACS40123 Fall24 Frequent Itemsets Tutorial](https://github.com/macs40123-f24/course-materials/blob/main/in-class-activities/02_pagerank_association/freq_itemset.ipynb).

**Compatibility**

| Platform                      | Compatible | Recommended | Notes                                                                                                                                    |
| ----------------------------- | ---------- | ----------- | ---------------------------------------------------------------------------------------------------------------------------------------- |
| **Local (e.g., 16GB laptop)** | ✅ Yes     | ✅ Yes      | 	Use the sampled dataset due to memory limits. |
| **Google Colab**              | ✅ Yes     | ✅ Yes  |  Limited compatibility with a sampled dataset.                                                    |
| **Midway3 Login Node**        | ✅ Yes      | ❌ No       | Not Recommended. Avoid running heavy computations on shared login nodes.    |
| **Midway3 Compute Node**      | ✅ Yes     | ✅ Yes      | Scales effectively; tested with 10 cores and 64GB memory on the `amd` partition. |

## Set up the Spark Environment

We will begin by creating a SparkSession and defining a small sample dataset, where each row represents a "basket" (or transaction) containing a list of items.

In this example, we will use a **minimum support count** of `s = 3`, mining itemsets that appear in at least 3 different baskets.

In [ ]:
import os

from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from itertools import combinations

# Initialize Spark Session
spark = SparkSession.builder.appName("FrequentItemsetMining").getOrCreate()

In [ ]:
# Sample data representing 8 different baskets
# Note that this is a toy example for demonstration purpose only. In real-world text representation, please remember to remove stopwords first.
data = [
    (0, ['cat', 'and', 'dog', 'bits']),
    (1, ['yahoo', 'news', 'claims', 'a', 'cat', 'mated', 'with', 'dog', 'and', 'produced', 'viable', 'offspring']),
    (2, ['cat', 'killer', 'likely', 'is', 'a', 'big', 'dog']),
    (3, ['professional', 'free', 'advice', 'on', 'dog', 'training', 'puppy']),
    (4, ['cat', 'and', 'kitten', 'training', 'behavior']),
    (5, ['dog', '&', 'cat', 'provides', 'training', 'in', 'eugene', 'oregon']),
    (6, ['dog', 'and', 'cat', 'is', 'a', 'slang', 'term', 'used', 'by', 'police', 'officers', 'for', 'male-female', 'relationship']),
    (7, ['shop', 'for', 'your', 'show', 'dog', 'grooming', 'and', 'pet', 'supplies'])
]


df = spark.createDataFrame(data, ["id", "basket"])
df.show(truncate=False)

+---+-----------------------------------------------------------------------------------------------+
|id |basket                                                                                         |
+---+-----------------------------------------------------------------------------------------------+
|0  |[cat, and, dog, bits]                                                                          |
|1  |[yahoo, news, claims, a, cat, mated, with, dog, and, produced, viable, offspring]              |
|2  |[cat, killer, likely, is, a, big, dog]                                                         |
|3  |[professional, free, advice, on, dog, training, puppy]                                         |
|4  |[cat, and, kitten, training, behavior]                                                         |
|5  |[dog, &, cat, provides, training, in, eugene, oregon]                                          |
|6  |[dog, and, cat, is, a, slang, term, used, by, police, officers, for, male-fem

---

## The A Priori Algorithm: A Step-by-Step Walkthrough

The A Priori algorithm identifies frequent itemsets through an interative process -- starting with individual items and progressively expanding to larger sets. Its key insight, the **A Priori Principle**, is simple but powerful:

> **If an itemset is frequent, then all of its subsets must also be frequent.**

This principal enables significant pruning of candidate itemsets. For example, if `{bread}` is not a frequent item, there's no need to calculate the support for `{bread, butter}`, since it cannot be frequent either.

### Step 1: Find Frequent Singletons (L₁)

We start by counting the frequency of every individual item and only record those that meet the minimum support threshold of `s=3`.


In [ ]:
# Set the support threshold
s = 3

# 1. flatMap to create a (item, 1) pair for each item in each basket
# 2. reduceByKey to sum the counts for each item
# 3. filter to keep only items with a count >= s
L1 = (
    df.rdd.flatMap(lambda row: [((item,), 1) for item in row.basket])
    .reduceByKey(lambda x, y: x + y)
    .filter(lambda x: x[1] >= s)
    .collect()
)

# We store frequent itemsets in a dictionary for easy lookup
Lk_1 = dict(L1)
print("Frequent Singletons (L1):")
print(Lk_1)

Frequent Singletons (L1):
{('cat',): 6, ('a',): 3, ('training',): 3, ('and',): 5, ('dog',): 7}


### Step 2: Find Frequent Pairs (L₂)

The next step is to find frequent pairs:

1.  **Generate Candidates (C₂):** Create all possible pairs from each basket.
2.  **Prune:** Keep only those candidate pairs where _both_ individual items are frequent singletons (in L₁). This is the A Priori principle in action\!
3.  **Count & Filter:** Count the support for the selected candidates and keep the frequent ones.

<!-- end list -->


In [ ]:
# Lk_1 is our dictionary of frequent singletons from the previous step

# 1. flatMap to generate all 2-item combinations from each basket
# 2. map to create a (pair, subsets) tuple. e.g., (('cat', 'dog'), {('cat',), ('dog',)})
# 3. filter to keep only pairs whose subsets are all in Lk_1
# 4. map to prepare for counting: (('cat', 'dog'), 1)
# 5. reduceByKey to count and filter to find L2
C2 = (
    df.rdd.flatMap(lambda row: [i for i in combinations(row.basket, 2)])
    .map(lambda pair: (pair, {tuple(sorted(i)) for i in combinations(pair, 1)}))
    .filter(lambda x: x[1].issubset(Lk_1.keys()))
    .map(lambda x: (tuple(sorted(x[0])), 1))
)

L2 = C2.reduceByKey(lambda x, y: x + y).filter(lambda x: x[1] >= s).collect()

L2 = dict(L2)
print("Frequent Pairs (L2):")
print(L2)

Frequent Pairs (L2):
{('and', 'dog'): 4, ('a', 'cat'): 3, ('and', 'cat'): 4, ('cat', 'dog'): 5, ('a', 'dog'): 3}


### Step 3: Find Frequent Triplets (L₃)

We repeat the process for 3-item sets, using the frequent pairs (L₂) to prune the candidates.


In [ ]:
# L2 is our dictionary of frequent pairs from the previous step

C3 = (
    df.rdd.flatMap(lambda row: [i for i in combinations(row.basket, 3)])
    .map(
        lambda triplet: (triplet, {tuple(sorted(i)) for i in combinations(triplet, 2)})
    )
    .filter(lambda x: x[1].issubset(L2.keys()))
    .map(lambda x: (tuple(sorted(x[0])), 1))
)
L3 = C3.reduceByKey(lambda x, y: x + y).filter(lambda x: x[1] >= s).collect()

L3 = dict(L3)
print("Frequent Triplets (L3):")
print(L3)

Frequent Triplets (L3):
{('and', 'cat', 'dog'): 3, ('a', 'cat', 'dog'): 3}


### Bringing it all Together: define the `apriori` algorithm

We can consolidate this logic into a single function.


In [ ]:
def apriori(df, support=3):
    """
    A basic implementation of the A Priori algorithm in PySpark.
    """
    # Find frequent singletons
    L1 = (
        df.rdd.flatMap(lambda x: [((i,), 1) for i in x.basket])
        .reduceByKey(lambda x, y: x + y)
        .filter(lambda x: x[1] >= support)
        .collect()
    )
    Lk_1 = dict(L1)
    freq_itemsets = (
        Lk_1.copy()
    )  # Use .copy() to avoid modifying the dict while iterating

    # Find frequent doubletons and higher
    k = 2
    while Lk_1:
        Lk = (
            df.rdd.flatMap(lambda x: [i for i in combinations(x.basket, k)])
            .map(lambda x: (x, set([tuple(sorted(i)) for i in combinations(x, k - 1)])))
            .filter(lambda x: set(x[1]).issubset(Lk_1.keys()))
            .map(lambda x: (tuple(sorted(x[0])), 1))
            .reduceByKey(lambda x, y: x + y)
            .filter(lambda x: x[1] >= support)
            .collect()
        )

        if not Lk:  # Break if no new frequent itemsets are found
            break

        Lk_1 = dict(Lk)
        freq_itemsets.update(Lk_1)
        k += 1

    return freq_itemsets


# Run the function and see the final result
all_freq_itemsets = apriori(df, support=3)
print("All Frequent Itemsets:")
print(all_freq_itemsets)

# Note: itemsets are represented with tuple data structure, no matter how many elements are contained in the itemset.

All Frequent Itemsets:
{('cat',): 6, ('a',): 3, ('training',): 3, ('and',): 5, ('dog',): 7, ('and', 'dog'): 4, ('a', 'cat'): 3, ('and', 'cat'): 4, ('cat', 'dog'): 5, ('a', 'dog'): 3, ('and', 'cat', 'dog'): 3, ('a', 'cat', 'dog'): 3}


---

## From Frequent Itemsets to Association Rules

Once we've identified the frequent itemsets, we can generate association rules in the form of  A→B , where A is the **antecedent** and B is the **consequent**, and we will evaluate the strength of each rule using **confidence** and **lift**.

- **Confidence**: The likelihood of observing item B when item A is present. A confidence of 80% for `chips -> salsa` means that 80% of customers who bought chips also bought salsa.

  $$Confidence(A \to B) = \frac{Support(A \cup B)}{Support(A)}$$

- **Lift**: Measures how much **more** likely we will observe item B when item A is present, compared to the baseline probability of B occurring on its own.


    $$Lift(A \to B) = \frac{Support(A \cup B)}{Support(A) \times Support(B)} $$


  - `Lift > 1`: A and B have a positive correlation (they appear together more than expected).
  - `Lift < 1`: A and B have a negative correlation.
  - `Lift = 1`: A and B are independent.


In [ ]:
def generate_association_rules(freq_itemsets, total_baskets, confidence_threshold=0.8):
  """
  Calculate association rules based on the confidence_threshold.
  """
    rules = []
    for itemset, support in freq_itemsets.items():
        if len(itemset) < 2:
            continue

        # Generate all non-empty proper subsets of the itemset
        subsets = [s for i in range(1, len(itemset)) for s in combinations(itemset, i)]

        for antecedent in subsets:
            antecedent = tuple(sorted(antecedent))
            consequent = tuple(sorted(set(itemset) - set(antecedent)))

            # This check is crucial
            if antecedent in freq_itemsets and consequent in freq_itemsets:
                antecedent_support = freq_itemsets.get(antecedent, 0)

                if antecedent_support > 0:
                    confidence = support / antecedent_support

                    if confidence >= confidence_threshold:
                        # Calculate lift
                        consequent_support = freq_itemsets.get(consequent, 0)
                        lift = (support * total_baskets) / (
                            antecedent_support * consequent_support
                        )
                        rules.append(((antecedent, consequent, confidence, lift)))

    return rules


# Get the total number of baskets
total_baskets = df.count()

# Generate and print the rules
association_rules = generate_association_rules(
    all_freq_itemsets, total_baskets, confidence_threshold=0.8
)

# Print in a readable format
print(f"{'Antecedent':<20} -> {'Consequent':<15} | {'Confidence':<12} | {'Lift'}")
print("-" * 65)
for rule in association_rules:
    ant, con, conf, lift = rule
    print(f"{str(ant):<20} -> {str(con):<15} | {conf:<12.3f} | {lift:.3f}")

Antecedent           -> Consequent      | Confidence   | Lift
-----------------------------------------------------------------
('and',)             -> ('dog',)        | 0.800        | 0.914
('a',)               -> ('cat',)        | 1.000        | 1.333
('and',)             -> ('cat',)        | 0.800        | 1.067
('cat',)             -> ('dog',)        | 0.833        | 0.952
('a',)               -> ('dog',)        | 1.000        | 1.143
('a',)               -> ('cat', 'dog')  | 1.000        | 1.600
('a', 'cat')         -> ('dog',)        | 1.000        | 1.143
('a', 'dog')         -> ('cat',)        | 1.000        | 1.333


This reveals a strong rule: customers who "buy" `('a', 'dog')` also "buy" `('cat',)` with 100% confidence.

---

## **Scalability Issues** of the A Priori Implementation ⚠️

While the self-defined apriori function works for small datasets, it is unsuitable to scale to large datasets for the following reasons:

1.  **Candidate Generation Bottleneck**: The step `flatMap(lambda x: [i for i in combinations(x.basket, k)])` is incredibly expensive. It generates a massive number of candidate itemsets on the Spark executors for every pass.
2.  **Multiple Data Passes**: The algorithm requires a full pass over the entire dataset for each `k` (i.e., for pairs, triplets, etc.). For large datasets, this I/O is very slow.
3.  **Driver Memory Overload**: The `.collect()` action at each step brings the _entire list of frequent k-itemsets_ back to the driver node. If you have millions of frequent pairs, this will easily crash the driver with an OutOfMemoryError. This is a classic Spark anti-pattern.

To fix this, one would need a much more complex implementation that keeps candidate sets distributed on the cluster. But there's a better way\!

### A Better Way: **FP-Growth** 🚀

Fortunately, we don't need to write our own algorithms. Spark MLlib has a parallel and highly scalable implementation of **FP-Growth**, which avoids the candidate generation bottleneck of A Priori. It uses a clever tree-based data structure (an FP-Tree) to find frequent itemsets in just two passes over the data.

Let's apply it on our small dataset first.


In [ ]:
from pyspark.ml.fpm import FPGrowth

# FPGrowth requires the input column to be named 'items'
fp_growth = FPGrowth(itemsCol="basket", minSupport=s / total_baskets, minConfidence=0.8)
model = fp_growth.fit(df)

# Display frequent itemsets
print("Frequent Itemsets found by FP-Growth:")
model.freqItemsets.sort("freq", ascending=False).show()

# Display generated association rules
print("Association Rules found by FP-Growth:")
model.associationRules.sort("confidence", ascending=False).show()

Frequent Itemsets found by FP-Growth:
+---------------+----+
|          items|freq|
+---------------+----+
|          [dog]|   7|
|          [cat]|   6|
|          [and]|   5|
|     [cat, dog]|   5|
|     [and, cat]|   4|
|     [and, dog]|   4|
|     [training]|   3|
|[and, cat, dog]|   3|
|            [a]|   3|
|       [a, cat]|   3|
|  [a, cat, dog]|   3|
|       [a, dog]|   3|
+---------------+----+

Association Rules found by FP-Growth:
+----------+----------+------------------+------------------+-------+
|antecedent|consequent|        confidence|              lift|support|
+----------+----------+------------------+------------------+-------+
|  [a, cat]|     [dog]|               1.0|1.1428571428571428|  0.375|
|  [a, dog]|     [cat]|               1.0|1.3333333333333333|  0.375|
|       [a]|     [cat]|               1.0|1.3333333333333333|  0.375|
|       [a]|     [dog]|               1.0|1.1428571428571428|  0.375|
|     [cat]|     [dog]|0.8333333333333334|0.9523809523809524|  0.

As you can see, the built-in `FPGrowth` model finds the same itemsets and rules with much less code and greater efficiency.

---

## Real-World Case Study: Analyzing Instacart Orders 🥑🍓🍌

Now let's apply FP-Growth to a large real-world dataset containing over 3 million Instacart orders. The goal is to discover food purchasing patterns that may reveal insights into recipes, dietary habits, or broader cultural trends.

_Note: In this demo, we only sample 1% of the data to ensure it runs quickly in a standard environment._


In [ ]:
# If you are using Midway, run this code
# DATA_DIR = "../datasets/instacart-3m"

In [ ]:
# If you are using Google Colab, run this code
import kagglehub

# Download latest version
path = kagglehub.dataset_download("psparks/instacart-market-basket-analysis")

DATA_DIR = path
print("Path to dataset files:", path)

Path to dataset files: /root/.cache/kagglehub/datasets/psparks/instacart-market-basket-analysis/versions/1


In [ ]:
# Load the data
# Make sure the CSV files are accessible in your environment
instacart_orders = spark.read.csv(
    os.path.join(DATA_DIR, "order_products__prior.csv"), header=True, inferSchema=True
)
product_codes = spark.read.csv(
    os.path.join(DATA_DIR, "products.csv"), header=True, inferSchema=True
)

# Aggregate products into baskets by order_id
instacart_baskets = instacart_orders.groupBy("order_id").agg(
    F.collect_list("product_id").alias("items")
)

# For demo purposes, sample down to 1% of the data
instacart_baskets_sample = instacart_baskets.sample(0.01, seed=40123)
instacart_baskets_sample.cache()  # Cache for faster access

# Configure and run FP-Growth
# Support = 3.2m orders * 1% sample * 0.001 support = ~32 orders
fp = FPGrowth(itemsCol="items", minSupport=0.001, minConfidence=0.5)
fpm = fp.fit(instacart_baskets_sample)

### Discovery 1: What are the most frequently purchased items?


In [ ]:
# Join with product names for readability
# This might take a minute
freq_items_with_names = fpm.freqItemsets.join(
    product_codes, fpm.freqItemsets.items[0] == product_codes.product_id
)
print("Top 10 Most Frequent Items:")
freq_items_with_names.sort("freq", ascending=False).show(10, truncate=False)

Top 10 Most Frequent Items:
+-------+----+----------+----------------------+--------+-------------+
|items  |freq|product_id|product_name          |aisle_id|department_id|
+-------+----+----------+----------------------+--------+-------------+
|[24852]|4806|24852     |Banana                |24      |4            |
|[13176]|3845|13176     |Bag of Organic Bananas|24      |4            |
|[21137]|2623|21137     |Organic Strawberries  |24      |4            |
|[21903]|2472|21903     |Organic Baby Spinach  |123     |4            |
|[47209]|2161|47209     |Organic Hass Avocado  |24      |4            |
|[47766]|1714|47766     |Organic Avocado       |24      |4            |
|[47626]|1521|47626     |Large Lemon           |24      |4            |
|[26209]|1419|26209     |Limes                 |24      |4            |
|[16797]|1405|16797     |Strawberries          |24      |4            |
|[27966]|1357|27966     |Organic Raspberries   |123     |4            |
+-------+----+----------+-----------

### Discovery 2: What purchasing patterns could we discover?

Let's review the generated association rules and join them with product names.


In [ ]:
# Join rules with product names
rules = fpm.associationRules

# 1. Create a lookup table for antecedent names
# Explode the array of product IDs, join with product names, then group back to a list of names
ant_lookup = rules.select("antecedent").distinct() \
    .withColumn("product_id", F.explode("antecedent")) \
    .join(product_codes, "product_id") \
    .groupBy("antecedent") \
    .agg(F.collect_list("product_name").alias("antecedent_name"))

# 2. Create a lookup table for consequent names
con_lookup = rules.select("consequent").distinct() \
    .withColumn("product_id", F.explode("consequent")) \
    .join(product_codes, "product_id") \
    .groupBy("consequent") \
    .agg(F.collect_list("product_name").alias("consequent_name"))

# 3. Join the original rules with both lookup tables
rules_with_names = rules.join(ant_lookup, "antecedent") \
    .join(con_lookup, "consequent") \
    .select("antecedent_name", "consequent_name", "confidence", "lift")

print("High-Confidence Association Rules (Lift > 2) with Full Item Lists:")
rules_with_names.filter("lift > 2").sort("confidence", ascending=False).show(n=30, truncate=False)

High-Confidence Association Rules (Lift > 2) with Full Item Lists:
+--------------------------------------------------------------------------------------------------------+-------------------------------------------------------+------------------+------------------+
|antecedent_name                                                                                         |consequent_name                                        |confidence        |lift              |
+--------------------------------------------------------------------------------------------------------+-------------------------------------------------------+------------------+------------------+
|[Non Fat Raspberry Yogurt, Vanilla Skyr Nonfat Yogurt]                                                  |[Icelandic Style Skyr Blueberry Non-fat Yogurt]        |0.6938775510204082|117.95918367346938|
|[Sparkling Water Berry, Lime Sparkling Water]                                                           |[Sparkling Water Grapef

We can observe rules that reveal common purchasing patterns. For example, customers who buy `Raspberry Yogurt` and `Vanilla Yogurt` are also very likely to buy `Blueberry Yogurt`.

### Discovery 3: The "Organic Fruit Bowl" Grammar

Looking more closely, one of the high-lift rules is `{Organic Hass Avocado, Organic Raspberries, Organic Strawberries} -> {Bag of Organic Bananas}`. This is like uncovering the _grammar_ of an organic fruit salad!

## Conclusion

This tutorial walked through the process of manually implementing the A Priori algorithm to leveraging Spark's highly optimized FP-Growth model.

**Key Takeaways:**

- **A Priori** provides the fundamental understanding of how frequent itemset mining works, but its candidate generation process limits scalability.
- **FP-Growth** is the industry-standard, scalable approach for market basket analysis in Spark, efficiently processing massive datasets in just two passes.
- Applying these techniques to real-world data can reveal compelling and actionable insights into consumer behavior -- from identifying common recipe combinations to understanding broader dietary trends.
